# Tutorial 1 - A simple pipeline to plot a signal

Here we define a simple custom signal agent pipeline. One agent will generate a
sine signal (in our case based on the built-in *SineGenerator*). The second one plots
the signals on the dashboard. Every agent of type *MonitorAgent* and
*MetrologicalMonitorAgent* l.

We define a *SineGeneratorAgent* for which we have to override the 
functions `init_parameters()` & `agent_loop()` to define the new agent's behaviour.

*   `init_parameters()` is used to setup the input data stream and potentially other 
necessary parameters.
*   `agent_loop()` will be endlessly repeated until further notice. It will 
sample by sample extract the input data stream's content and push it to all agents 
connected to *SineGeneratorAgent*'s output channel by invoking `send_output()`.

The *MonitorAgent* is connected to the *SineGeneratorAgent*'s output channel and per 
default automatically plots the output. 

Each agent has an internal `current_state` which can be used as a switch to change the 
behaviour of the agent. The available states are listed
[here](https://github.com/bangxiangyong/agentMET4FOF/blob/a95b788544e8cce1e0bb757184da8c6447e96927/agentMET4FOF/agents.py#L78).

As soon as all agents are initialized and the connections are set up, the agent 
network is started by accordingly changing all agents' state simultaneously.

In [2]:
# %load simple_generator.py
from agentMET4FOF.agents import AgentMET4FOF, AgentNetwork, MonitorAgent
from agentMET4FOF.streams import DataStreamMET4FOF
import numpy as np


class SineGenerator(DataStreamMET4FOF):
    """Copy of the built-in class of a streaming sine wave generator

    `sfreq` is sampling frequency which determines the time step when next_sample is called
    `F` is frequency of wave function
    `sine_wave_function` is a custom defined function which has a required keyword `time` as argument and any number of optional additional arguments (e.g `F`).
    to be supplied to the `set_generator_function`

    """

    def __init__(self, sfreq=500, F=5):
        super().__init__()
        self.set_metadata(
            "SineGenerator",
            "time",
            "s",
            ("Voltage"),
            ("V"),
            "Simple sine wave generator",
        )
        self.set_generator_function(
            generator_function=self.sine_wave_function, sfreq=sfreq, F=F
        )

    def sine_wave_function(self, time, F=50):
        value = np.sin(2 * np.pi * F * time)
        return value


class SineGeneratorAgent(AgentMET4FOF):
    """An agent streaming a sine signal

    Takes samples from the :py:mod:`SineGenerator` and pushes them sample by sample
    to connected agents via its output channel.
    """

    # # The datatype of the stream will be SineGenerator.
    _sine_stream: SineGenerator

    def init_parameters(self):
        """Initialize the input data

        Initialize the input data stream as an instance of the
        :py:mod:`SineGenerator` class
        """
        self.sine_stream = SineGenerator()

    def agent_loop(self):
        """Model the agent's behaviour

        On state *Running* the agent will extract sample by sample the input data
        streams content and push it via invoking :py:method:`AgentMET4FOF.send_output`.
        """
        if self.current_state == "Running":
            sine_data = self.sine_stream.next_sample()  # dictionary
            self.send_output(sine_data["quantities"])


def demonstrate_generator_agent_use():
    # Start agent network server.
    agent_network = AgentNetwork()

    # Initialize agents by adding them to the agent network.
    gen_agent = agent_network.add_agent(agentType=SineGeneratorAgent)
    gen_agent.init_parameters()
    monitor_agent = agent_network.add_agent(agentType=MonitorAgent)

    # Interconnect agents by either way:
    # 1) by agent network.bind_agents(source, target).
    agent_network.bind_agents(gen_agent, monitor_agent)

    # 2) by the agent.bind_output().
    gen_agent.bind_output(monitor_agent)

    # Set all agents' states to "Running".
    agent_network.set_running_state()

    # Allow for shutting down the network after execution
    return agent_network


if __name__ == "__main__":
    demonstrate_generator_agent_use()


Error on connecting to existing name server at http://0.0.0.0:3333: Could not locate the name server!
Starting NameServer...
Broadcast server running on 0.0.0.0:9091
NS running on 0.0.0.0:3333 (0.0.0.0)
URI = PYRO:Pyro.NameServer@0.0.0.0:3333
INFO [2021-07-30 11:30:13.395180] (SineGeneratorAgent_1): INITIALIZED
INFO [2021-07-30 11:30:13.437144] (MonitorAgent_1): INITIALIZED
[2021-07-30 11:30:13.453633] (SineGeneratorAgent_1): Connected output module: MonitorAgent_1

|----------------------------------------------------------|
|                                                          |
| Your agent network is starting up. Open your browser and |
| visit the agentMET4FOF dashboard on http://0.0.0.0:8050/ |
|                                                          |
|----------------------------------------------------------|

SET STATE:   Running
[2021-07-30 11:30:14.400304] (SineGeneratorAgent_1): Pack time: 0.00028
[2021-07-30 11:30:14.400995] (SineGeneratorAgent_1): Sending: [0.]
[2